In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
import random
%matplotlib inline


In [3]:
# !pip install numpy


In [95]:
class Value:
  
  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None # this is the None version for functions. So a function is stored here, it executes when called later with an object
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"
  
  def __add__(self, other):
    other = other if isinstance(other, Value) else Value(other) #converts int to Value in cases of Value(2) + 2, technically other is an int and other.data would be undefined
    out = Value(self.data + other.data, (self, other), '+')
    
    def _backward():
      self.grad += 1.0 * out.grad # this line accumulates grasdients for repeated variables. eg. if 'a' branches to b and c, back prop from b would update a.grad and back prop from c would overide it. Mathematically, these two should accumulate to get the global gradient
      other.grad += 1.0 * out.grad
    out._backward = _backward
    
    return out
  
  def __radd__(self, other):
    """
    a = Value(2)
    2+a
    to check bugs in expressions above. Python falls back to this method and reverses the order of the elements since the 2 was not a Value type.
    """
    return Value(self.data + other)

  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data, (self, other), '*')

    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward
      
    return out

  def __rmul__(self, other):
    """
    a = Value(2)
    2*a
    to check bugs in expressions above. Python falls back to this method and reverses the order of the elements since the 2 was not a Value type.
    """
    return Value(self.data * other)

  def __truediv__(self, other): # self / other
    return self * other**-1 # for divion but a/b -> a*(1/b) = a*b**-1

  def __neg__(self): # -self
    return self * -1

  def __sub__(self, other): # self - other
    return self + (-other) # the addition operator already defined is used here but negation introduced. This is done immediately above

  def __radd__(self, other): # other + self
    return self + other
  
  def tanh(self):
    x = self.data
    t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
    out = Value(t, (self, ), 'tanh')
    
    def _backward():
      self.grad += (1 - t**2) * out.grad
    out._backward = _backward
    
    return out
  
  def exp(self):
    x = self.data
    out = Value(math.exp(x), (self, ), 'exp')
    
    def _backward():
      self.grad += out.data * out.grad
    out._backward = _backward
    
    return out
  
  def backward(self):
    """
    automated back prop from the given node.
    Takes a node
    Stretches it out to get all chindren node
    Starting from the last node, for each child node it calls node.backward() 
    this executes the function defined to update the gradients and populates the self.grad for all children(nodes)
    """
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)
    
    self.grad = 1.0 # this initialises end node gradient to itself to be zero in order to start the back prop properly
    for node in reversed(topo): # populates gradients based on the _backward function for each operation[addition, multiplication, tanh]
      node._backward()




class Neuron:
  
  def __init__(self, nin):
    # nin is the number if inputs to the neuron
    self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
    self.b = Value(random.uniform(-1,1))
  
  def __call__(self, x):
    # w * x + b
    assert(len(self.w) == len(x)), "Input shape do not match input shape of neuron"
    print((wi*xi for wi, xi in zip(self.w, x)), self.b)
    act = value(1)
    print("act here", act)
    # act = x * self.b

    out = act.tanh()
    return out
  
  def parameters(self):
    return self.w + [self.b]

class Layer:
  
  def __init__(self, nin, nout):
    self.neurons = [Neuron(nin) for _ in range(nout)]
  
  def __call__(self, x):
    outs = [n(x) for n in self.neurons]
    return outs[0] if len(outs) == 1 else outs
  
  def parameters(self):
    return [p for neuron in self.neurons for p in neuron.parameters()]

class MLP:
  
  def __init__(self, nin, nouts):
    sz = [nin] + nouts
    self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]
  
  def __call__(self, x):
    for layer in self.layers:
      x = layer(x)
    return x
  
  def parameters(self):
    return [p for layer in self.layers for p in layer.parameters()]

In [96]:
x = [2,3]
n = Neuron(2)
print(n.w)
print(n.b)
n(x)

[Value(data=-0.3936961612468821), Value(data=0.7082014846439535)]
Value(data=-0.3836765653390257)
<generator object Neuron.__call__.<locals>.<genexpr> at 0x7a70ddaaa7a0> Value(data=-0.3836765653390257)


NameError: name 'value' is not defined

In [76]:
[1,2].len()

AttributeError: 'list' object has no attribute 'len'